# INTELIGENCIA ARTIFICIAL: EB Ejercicios Tema 4: REGRESIÓN LOGÍSTICA 
## EJERCICIO: ¿Cómo resolver un problema multiclase?
**Implementar** una regresión logística para reconocer números escritos a mano del 1 al 10. Use como método de evaluación holdout, donde el conjunto de entrenamiento está formado por el 70% de las primeras filas del conjunto de datos, y el conjunto de test por el resto.

Para ello, tenemos un conjunto de datos formado por 5000 ejemplos de dígitos escritos a mano en **ex4data1.mat**. La extensión .mat indica que contiene datos salvados en formato matriz Octave/Matlab nativo en vez de en formato texto. Una vez cargado el fichero .mat, tendrás en memoria la variable **X** que contiene la matriz de atributos, y el vector **y** que contiene las etiquetas.

Después de un preprocesado de las imágenes formadas por los dígitos escritos a mano, cada imagen se codifica como un vector de 400 elementos, donde cada elemento es un número decimal que representa la intensidad de gris en una posición determinada. Las etiquetas son 1, 2, 3, 4, 5, 6, 7, 8, 9, 0 donde la etiqueta 0 representa el dígito 10.

In [1]:
import numpy as np
import scipy.io as sio
import scipy.optimize as op
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics

# A) EJERCICIO BASE

## 1) Cargar los datos de entrada

In [2]:
print("Cargando los datos ...\n")
data = sio.loadmat("ex4data1.mat") # dict type

X = pd.DataFrame(data['X'])
y = pd.DataFrame(data['y'])
m = X.shape[0]

print("El tamaño de X es: ", X.shape)
print("La longitud del vector y es: ", len(y))

Cargando los datos ...

El tamaño de X es:  (5000, 400)
La longitud del vector y es:  5000


## 2) Hipótesis y función coste
En regresión logística el modelo viene definido por: 
\begin{equation}
h_{\theta}(X) = g(X\theta)
\end{equation}

siendo  

\begin{equation}
g(z) = \frac{1}{1+e^{-z}}
\end{equation}

El objetivo de la regresión logística es minimizar el coste de la función:

\begin{equation}
J(\theta)=\frac{-1}{m}\sum_{i=1}^{m} (y^i \cdot log(h_{\theta}(x^i))+(1-y^i)\cdot log(1-h_{\theta}(x^i)))
\end{equation}

\begin{equation}
= 
\frac{-1}{m}\sum (y \cdot log(h_{\theta}(X))+(1-y)\cdot log(1-h_{\theta}(X)))
\end{equation}

In [3]:
def sigmoid(z):
    g = 1 / (1+np.exp(-z)) 

    return g

def costFunction(theta, X, y):
    m = len(y)
    h = sigmoid(np.dot(X, theta))
    J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))

    return J

#### 2.1) Añadir $x_0$
x0 se añade como primera columna a X con todos sus elementos a 1.

In [4]:
X.insert(0, 'ones', 1) # Se crea una nueva columna 'ones' en la primera columna (primer parámetro:0) con todos sus valores a 1 (tercer parámetro:1)

#### 2.2) Inicializar $\theta$

In [5]:
num_atributos = X.shape[1] # Si esta operación la hacemos antes de añadir la columna de 1 a X, debemos poner X.shape[1]+1
initial_theta = np.zeros((num_atributos,1), dtype=np.float64)

## 3) Descenso del Gradiente para optimización
Con el objeto de usar la función avanzada de optimización de python fmin_cg, se va a usar la función gradientFunction_optimization que devuelve el valor del gradiente formado por las derivadas parciales. Las derivadas parciales se calculan como:
\begin{equation}
\frac{\partial J(\theta)}{\partial \theta_j}=\frac{1}{m}\sum_{i=1}^{m} (h_{\theta}(x^i)-y^i)\cdot x_j^i
\end{equation}
Recordar que para que la programación sea eficiente el gradiente se debe vectorizar.


In [6]:
def gradientFunction_optimization(theta, X, y):
    m = len(y)
    
    h = sigmoid(np.dot(X, theta))
    grad = (1/m) * (np.dot(X.T,(h-y))) # Recuerde: grad debería tener las mismas dimensiones que theta

    return grad

### 3.1) Función que transforma la salida (multiclase) para poder usarse en problemas de clasificación binaria.
Para ello hay que implementar una función que reciba como parámetros: la salida ($y$) y el valor de la clase actual (de 1 a 10 en este caso) que se esté usando en el problema de clasificación binaria. La salida de la función tendrá tantas filas como tuviese $y$ con valor 1 cuando la clase que se pasa por parámetro coincida con el valor de $y$ en esa fila y con 0 en caso contrario.

In [7]:
# COMPLETAR
# OPCION 1
def y_change(y, cl):
  return pd.get_dummies(y.to_numpy().flatten())[cl]  # One-hot enconding de y seleccionando la columna de la clase cl

# OPCION 2
def y_change1(y, cl):
    y_pr=[] # Lista vacía para ir añadiendo 0 o 1 según si el valor de la fila de y es cl o no
    for i in range(0, len(y)): # Recorro todas las filas de y
        if y[0][i] == cl: # y tiene una columna que se llama 0 y las filas las recorremos con i
            y_pr.append(1) # Añado 1 a la lista si el valor de la fila i es cl
        else:
            y_pr.append(0) # Añado 0 a la lista si el valor de la fila i no es cl
    y_new = pd.DataFrame({'label':y_pr})
    return y_new

### 3.2) Función para entrenar problemas de clasificación multiclase
Implementar una función llamada training que sea la encargada de realizar el entrenamiento del método. Para ello, esta función recibirá el número de clases, los parámetros theta iniciales y el conjunto de entrenamiento (X_train,y_train), y devolverá los parámetros theta que definen el modelo aprendido. Las dimensiones de estos son num_class x (n+1), donde n es el número de atributos de la matriz X.

Dado que se trata de un problema de clasificación multiclase, habría que reducir el problema a problemas de clasificación binaria que es lo que sabemos resolver. En concreto, a 10 problemas de clasificación binaria ya que tenemos 10 clases. En el entrenamiento de cada regresión logística hay que pasarle el conjunto de entrenamiento adecuado. 

In [8]:
# COMPLETAR
def training(initial_theta, X_train, y_train, num_clases):
  all_theta = [] # Lista vacía para incluir las theta óptimas de cada clase
  all_cost = [] # Lista vacía para incluir el coste final de cada clase
  all_class = [] # Lista vacía para añadir las clases

  for current_class in range(1, num_clases+1): # Vamos a aplicar el algoritmo de optimización a cada clase (desde 1 hasta 10 (se pone +1 para que se incluya el 10 en el for))
      res_optimization = op.fmin_cg(maxiter=150, f=costFunction, x0=initial_theta.flatten(), 
                                    fprime=gradientFunction_optimization,
                             args=(X, y_change(y, current_class).to_numpy().flatten()), 
                             full_output = True) # Indicando full_output=True la salida del algoritmo de optimización
                                # será en la posición [0] los theta óptimos y en la posición [1] el coste alcanzado por esos theta optimos

      all_theta.append(res_optimization[0]) # Theta óptimos del algoritmo de optimización
      all_cost.append(res_optimization[1]) # Coste del algoritmo de optimización
      all_class.append(current_class) # Clase actual
    
  df_opt = pd.DataFrame({'class':all_class, 'theta':all_theta, 'cost':all_cost}) # Creamos dataframe que devolverá la función
  
  return df_opt

## 4) Predecir en problemas multiclase
Tenga en cuenta que la predicción es la predicción que tiene una mayor probabilidad de entre todas las predicciones obtenidas por los 10 modelos. 

In [9]:
# COMPLETAR
def predict(res_optimization, X):
  m = len(X)
  all_instances = [] # Lista vacía para incluir las instancias/filas de X
  all_predictions = [] # Lista vacía para incluir las predicciones

  for i in range(0, len(X)): # Para cada fila de X
    all_h = [] # Lista vacía para añadir las h de la fila i (tamaño de all_h: num_classes en este caso 9)
  
    for current_class in res_optimization['class']: # Para los theta optimos de cada clase
      h = sigmoid(np.dot(X.iloc[i], res_optimization['theta'][current_class - 1]))
      all_h.append(h) # Añado cada h a la lista all_h
    
    pred = np.argmax(all_h)+1 # ¡¡IMPORTANTE!! Busco la mejor predicción. La función devuelve el índice de la mejor predicción (predicciones son de 1 a 10 y no de 0 a 9)
    all_predictions.append(pred) # Añado la mejor a la lista de predicciones finales
    all_instances.append(i) # Añado la instancia/fila a la lista
  
  return pd.DataFrame({'instance':all_instances, 'prediction':all_predictions})

def accuracy_model (y_real, y_predictions):
  return np.mean(y_predictions['prediction']==y_real[0]) # [0] es porque la columna del dataframe y_real se llama 0

# B) EVALUACIÓN USANDO HOLDOUT
Una vez tenemos todas las funciones necesarias, vamos a unir todas las piezas para implementar la función principal. El programa principal consta de las siguientes partes:


1. Inicializar los parámetros necesarios.
2. Dividir el dataset en 70% para entrenamiento y 30% para test usando la función holdout suministrada en EB Ejercicios T3.
3. Obtener todos los modelos y los valores del coste J a través de las iteraciones de todos los modelos usando la función training. 
4. Predecir el conjunto de training y calcular el error.
5. Predecir el conjunto de test y calcular el error.

In [10]:
# OPCION 2
def holdout_opcion2(X, y, percentage=0.6):
  X_training = X.sample(round(percentage*len(X))) # Selecciona aleatoriamente el numero de filas indicado
  y_training = y.iloc[X_training.index] # Selecciona las filas del X_training
  X_test = X.iloc[~X.index.isin(X_training.index)] # ~ significa NOT
  y_test = y.iloc[~X.index.isin(X_training.index)] # ~ significa NOT

  print("El tamaño del training debe ser: ", round(percentage*len(X)), " - Comprobación: tamaño X_training es ", len(X_training), " y tamaño y_training es", len(y_training))
  print("El tamaño del test debe ser: ", len(X)-round(percentage*len(X)), " - Comprobación: tamaño X_test es ", len(X_test), " y tamaño y_test es", len(y_test))

  # Reseteamos los índices de todos los conjuntos
  X_training = X_training.reset_index(drop=True)
  y_training = y_training.reset_index(drop=True)
  X_test = X_test.reset_index(drop=True)
  y_test = y_test.reset_index(drop=True)
  
  return X_training, y_training, X_test, y_test

In [11]:
# COMPLETAR
# Paso 1:
num_classes = 10
initial_theta = np.zeros((X.shape[1],1))
# Columna de 1s en la primera posición de X está ya hecho --> comprobar

# Paso 2: HOLDOUT 70-30
X_training, y_training, X_test, y_test = holdout_opcion2(X, y, 0.7)

# Paso 3: ENTRENAMIENTO
res_optimization_training = training(initial_theta, X_training, y_training, num_classes)

El tamaño del training debe ser:  3500  - Comprobación: tamaño X_training es  3500  y tamaño y_training es 3500
El tamaño del test debe ser:  1500  - Comprobación: tamaño X_test es  1500  y tamaño y_test es 1500


C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: divide by zero encountered in log
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: invalid value encountered in multiply
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: divide by zero encountered in log
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: invalid value encountered in multiply
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: divide by zero encountered in log
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: R

         Current function value: nan
         Iterations: 135
         Function evaluations: 653
         Gradient evaluations: 653


c:\Users\danit\anaconda3\envs\entornoIA2425\lib\site-packages\scipy\optimize\_optimize.py:1659: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


         Current function value: 0.042473
         Iterations: 150
         Function evaluations: 457
         Gradient evaluations: 457


...

c:\Users\danit\anaconda3\envs\entornoIA2425\lib\site-packages\scipy\optimize\_optimize.py:1659: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


         Current function value: 0.062925
         Iterations: 150
         Function evaluations: 416
         Gradient evaluations: 416


C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: divide by zero encountered in log
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: invalid value encountered in multiply
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: divide by zero encountered in log
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: invalid value encountered in multiply
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: divide by zero encountered in log
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: R

         Current function value: nan
         Iterations: 146
         Function evaluations: 1476
         Gradient evaluations: 1476


C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: divide by zero encountered in log
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: invalid value encountered in multiply
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: divide by zero encountered in log
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
C:\Users\danit\AppData\Local\Temp\ipykernel_13304\285225370.py:9: RuntimeWarning: invalid value encountered in multiply
  J = -(1 / m) * np.sum((y * np.log(h)) + ((1 - y) * np.log(1 - h)))
c:\Users\danit\anaconda3\envs\entornoIA2425\lib\site-packages\scipy\optimize\_optimize.py:1659: OptimizeWarning: NaN result encountered.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


In [12]:
# COMPLETAR
# Paso 4
res_prediction_training = predict(res_optimization_training, X_training) # PREDICCIÓN DEL TRAINING
accuracy_training = accuracy_model(y_training, res_prediction_training) # ACCURACY DE LA PREDICCIÓN DEL TRAINING
print("Training accuracy: ", accuracy_training)
print("Training accuracy sklearn:", metrics.accuracy_score(y_training,res_prediction_training['prediction']))

Training accuracy:  0.9731428571428572
Training accuracy sklearn: 0.9731428571428572


In [13]:
# COMPLETAR
# Paso 5
res_prediction_test = predict(res_optimization_training, X_test) # PREDICCIÓN DEL TEST
accuracy_test = accuracy_model(y_test, res_prediction_test) # ACCURACY DE LA PREDICCIÓN DEL TEST
print("Test accuracy: ", accuracy_test)
print("Test accuracy sklearn:", metrics.accuracy_score(y_test, res_prediction_test['prediction']))

Test accuracy:  0.9666666666666667
Test accuracy sklearn: 0.9666666666666667
